In [ ]:
%load_ext autoreload
%autoreload 2

## Setup


In [ ]:
# Add project root (parent directory) to system path (for module imports)
import sys

sys.path.append("../")

In [ ]:
import torch

from src.dataset import CocoDataset, split_coco_annotations
from src.models import ImageCaptioningModel, TransformerMappingNetwork
from src.train import train

In [ ]:
# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
# Seed
SEED = 42
# TODO: Set seed for each relevant library (torch, numpy, random, etc.)

## Prepare Datasets


In [ ]:
MAX_CAPTION_LENGTH = 50
EMBEDDINGS_PATH = "../data/coco/embeddings/"
ANNOTATIONS_PATH = "../data/coco/annotations/"
IMAGES_PATH = "../data/coco/val2017/"  # For FiftyOne visualization
CHECKPOINTS_PATH = "../checkpoints/"

In [ ]:
# We split the original COCO 2014 training set into a new training and validation set
split_coco_annotations(
    annotations_path=ANNOTATIONS_PATH + "captions_train2014.json",
    output_dir=ANNOTATIONS_PATH,
    split_ratio=0.8,
    seed=SEED,
)

In [ ]:
# Training Dataset (orig. COCO 2014 TRAIN)
train_dataset = CocoDataset(
    embeddings_path=EMBEDDINGS_PATH + "train_val_clip_embeddings.pt",
    annotations_path=ANNOTATIONS_PATH + "train_split.json",
    max_length=MAX_CAPTION_LENGTH,
    normalize_embeddings=False,  # `.pt` files already contain normalized embeddings
)

# Validation Dataset (orig. COCO 2014 TRAIN)
val_dataset = CocoDataset(
    embeddings_path=EMBEDDINGS_PATH + "train_val_clip_embeddings.pt",
    annotations_path=ANNOTATIONS_PATH + "val_split.json",
    max_length=MAX_CAPTION_LENGTH,
    normalize_embeddings=False,
)

# Test Dataset (orig. COCO 2017 Val)
test_dataset = CocoDataset(
    embeddings_path=EMBEDDINGS_PATH + "test_clip_embeddings.pt",
    annotations_path=ANNOTATIONS_PATH + "captions_val2017.json",
    max_length=MAX_CAPTION_LENGTH,
    normalize_embeddings=False,
)

## Prepare Model


In [ ]:
# Models
mapping_network = TransformerMappingNetwork(
    embed_dim=512,  # CLIP embedding dimension
    gpt_dim=768,  # GPT-2 embedding dimension
    prefix_length=40,
    hidden_length=40,
)

model = ImageCaptioningModel(
    mapping_network=mapping_network,
    freeze_gpt_weights=True,  # We only fine-tune the mapping network during training
).to(DEVICE)

print(model)

## Train Model (with Validation Evaluation)


In [ ]:
# Train image captioning model with validation evaluation
history = train(
    train_dataset=train_dataset,
    model=model,
    batch_size=64,
    num_epochs=1,
    device=DEVICE,
    outputs_dir=CHECKPOINTS_PATH,
    # Evaluation on validation set
    val_dataset=val_dataset,
    val_annotations_path=ANNOTATIONS_PATH + "val_split.json",
    eval_every_epoch=1,
)

print(
    f"\nBest validation CIDEr: {history['best_val_cider']:.4f} at epoch {history['best_epoch']}"
)

---

## Evaluation

Run evaluation on the test set after training is complete.


In [ ]:
# Import evaluation functions
from src.eval import compute_caption_metrics, evaluate_captions, evaluate_epoch

In [ ]:
# Load best model checkpoint (optional - if you want to evaluate a saved model)
# checkpoint_path = CHECKPOINTS_PATH + "best_model.pth"
# model.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE))
# print(f"Loaded checkpoint: {checkpoint_path}")


In [ ]:
# Evaluate on TEST set (final evaluation - only run once after training!)
test_metrics = evaluate_epoch(
    model=model,
    dataset=test_dataset,
    annotations_path=ANNOTATIONS_PATH + "captions_val2017.json",
    epoch=history["best_epoch"],
    split_name="test",
    batch_size=32,
    num_workers=4,  # Set to 0 on Windows
    device=DEVICE,
    output_dir=CHECKPOINTS_PATH + "eval_results",
)

print(f"\nFinal Test Results: {test_metrics}")

In [ ]:
# View generated captions
import json

predictions_path = (
    CHECKPOINTS_PATH
    + f"eval_results/epoch_{history['best_epoch']}_test_predictions.json"
)
with open(predictions_path) as f:
    predictions = json.load(f)

print(f"Generated {len(predictions)} captions\n")
print("Sample predictions:")
for pred in predictions[:10]:
    print(f"  Image {pred['image_id']}: {pred['caption']}")

---

## Quick Test: Evaluation Functions with Mock Data

Test the evaluation functions without needing real data.


In [ ]:
# Test compute_caption_metrics with mock data
print("Testing compute_caption_metrics()...")

mock_predictions = {
    1: ["a dog sitting on the grass"],
    2: ["a cat sleeping on a couch"],
    3: ["a person riding a bicycle"],
}

mock_references = {
    1: [
        "a dog is sitting on green grass",
        "dog on the grass",
        "a brown dog sits on grass",
    ],
    2: [
        "a cat is sleeping on the sofa",
        "cat napping on couch",
        "a sleeping cat on a couch",
    ],
    3: ["a man rides a bike", "person on a bicycle", "someone cycling on the road"],
}

metrics = compute_caption_metrics(mock_predictions, mock_references)
print(f"✅ Success! Metrics: {metrics}")

In [ ]:
# Test evaluate_captions with mock COCO format
import os
import tempfile

print("Testing evaluate_captions()...")

mock_coco = {
    "images": [
        {"id": 1, "file_name": "img1.jpg"},
        {"id": 2, "file_name": "img2.jpg"},
    ],
    "annotations": [
        {"image_id": 1, "id": 1, "caption": "a dog sitting on grass"},
        {"image_id": 1, "id": 2, "caption": "dog on the green grass"},
        {"image_id": 2, "id": 3, "caption": "a cat on a couch"},
        {"image_id": 2, "id": 4, "caption": "cat sleeping on sofa"},
    ],
}

mock_pred_list = [
    {"image_id": 1, "caption": "a dog sitting on the grass"},
    {"image_id": 2, "caption": "a cat sleeping on the couch"},
]

with tempfile.NamedTemporaryFile(mode="w", suffix=".json", delete=False) as f:
    json.dump(mock_coco, f)
    temp_path = f.name

try:
    metrics = evaluate_captions(mock_pred_list, temp_path)
    print(f"✅ Success! Metrics: {metrics}")
finally:
    os.unlink(temp_path)

---

## FiftyOne Visualization

Interactive visualization of generated captions vs. ground truth.


In [ ]:
# Import visualization functions
from src.visualize import create_captioning_dataset

print("FiftyOne visualization ready!")

In [ ]:
# Create FiftyOne dataset from evaluation results
dataset = create_captioning_dataset(
    images_dir=IMAGES_PATH,
    predictions_path=CHECKPOINTS_PATH
    + f"eval_results/epoch_{history['best_epoch']}_test_predictions.json",
    annotations_path=ANNOTATIONS_PATH + "captions_val2017.json",
    dataset_name="caption_eval",
)

print(dataset)

In [ ]:
# Browse samples in the dataset
for sample in dataset.take(5):
    print(f"\nImage ID: {sample.image_id}")
    print(f"Generated: {sample.generated_caption}")
    print(f"References: {sample.reference_captions[:2]}...")  # Show first 2

In [ ]:
# Launch FiftyOne app for interactive exploration
# This opens a web browser at http://localhost:5151

# Uncomment to launch:
# launch_app(dataset)

# Or launch without blocking:
# session = fo.launch_app(dataset)
# session.show()


---

## Plot Metrics Over Training


In [ ]:
import matplotlib.pyplot as plt

# Plot validation metrics from training history (if available)
if history["val_metrics"]:
    val_history = history["val_metrics"]
    epochs = [m["epoch"] for m in val_history]

    # Extract metrics
    bleu4 = [m.get("BLEU-4", 0) for m in val_history]
    cider = [m.get("CIDEr", 0) for m in val_history]
    meteor = [m.get("METEOR", 0) for m in val_history]

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(epochs, bleu4, "b-o", label="BLEU-4")
    ax.plot(epochs, cider, "r-s", label="CIDEr")
    ax.plot(epochs, meteor, "g-^", label="METEOR")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Score")
    ax.set_title("Validation Metrics Over Training")
    ax.legend()
    ax.grid(True)
    plt.show()
else:
    print("No validation metrics available - train for more epochs to see trends")

---

## Summary

### Evaluation Functions:
- `compute_caption_metrics(preds, refs)` - Low-level metric computation
- `evaluate_captions(predictions, annotations_path)` - Evaluate list of predictions  
- `generate_and_evaluate(model, dataset, ...)` - Generate + evaluate in one call
- `evaluate_epoch(model, dataset, ...)` - Full epoch eval with file saving

### Visualization Functions:
- `create_captioning_dataset(...)` - Build FiftyOne dataset
- `create_comparison_dataset(...)` - Compare multiple models
- `get_low_score_view(...)` / `get_high_score_view(...)` - Filter samples
- `launch_app(dataset)` - Interactive web visualization

### Metrics:
- **BLEU-1/2/3/4**: N-gram precision
- **METEOR**: Semantic matching with synonyms
- **CIDEr**: Consensus-based TF-IDF weighted (most important for captioning)
- **ROUGE-L**: Longest common subsequence
